In [21]:
import pandas as pd
from pycaret.classification import *
from sklearn.preprocessing import LabelEncoder
import plotly.express as px

from diskcache import Cache
from appdirs import user_cache_dir

cache = Cache(user_cache_dir("cache"))
GENERATED_DATASET_PATH = cache.get("GENERATED_DATASET_PATH")
df = pd.read_csv(f'{GENERATED_DATASET_PATH}/df.csv')
df

,curso,serie,tipo_residente,atv_cultural_apresentacoes musicais,atv_cultural_artesanato,atv_cultural_crio historia e enigmas,atv_cultural_danca,atv_cultural_edicoes de video...,atv_cultural_esporte,atv_cultural_fotografia,...,transporte_melhoria no preco da passagem,transporte_nenhuma opcao,projetos_a_participar_adicionar sessões de RPG.,projetos_a_participar_aula de volei para todos,projetos_a_participar_nenhuma opcao,projetos_a_participar_oficina de teclado,projetos_a_participar_oficina de violao,projetos_a_participar_radio do campus,projetos_a_participar_recuperacao e ocupacao do antigo cine januaria,projetos_a_participar_time de e-sports do campus e torneios intercampis.
0,agro,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,agro,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,agro,0,0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
3,agro,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,agro,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,mamb,2,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
607,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
608,mamb,2,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
609,mamb,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [22]:
enc = LabelEncoder()
df['curso'] = enc.fit_transform(df['curso'])

# Calcula a correlação
corr_matrix = df.corr(method='kendall')

# Remover missing values antes de continuar
corr_matrix = corr_matrix.dropna(axis=0, how='any').dropna(axis=1, how='any')

# Filtra as correlações
filtered_corr = corr_matrix[(corr_matrix > 0.5) | (corr_matrix < -0.5)]

In [71]:
# Pega apenas as correlações fortes da matriz original
corr_matrix_strong = corr_matrix[(corr_matrix > 0.5) | (corr_matrix < -0.5)].copy()

# Remove colunas e linhas totalmente vazias
corr_matrix_strong = corr_matrix_strong.dropna(axis=0, how='all').dropna(axis=1, how='all')

fig = px.imshow(
    corr_matrix_strong,
    text_auto=True,
    color_continuous_scale='RdBu',
    zmin=-1,
    zmax=1,
    aspect='auto'
)

fig.update_layout(
    xaxis_side='bottom',
    height=900
)

fig.show()

In [24]:
# Cria tabela de correlações
correlation_table = corr_matrix.unstack().reset_index()
correlation_table.columns = ['Feature 1', 'Feature 2', 'Correlation']

# Remover linhas onde as duas features são iguais (auto-correlação)
correlation_table = correlation_table[correlation_table['Feature 1'] != correlation_table['Feature 2']]

# Remover duplicatas e ordenar pela correlação
correlation_table['ordered_pair'] = correlation_table.apply(
    lambda row: tuple(sorted([row['Feature 1'], row['Feature 2']])), axis=1
)
correlation_table = correlation_table.drop_duplicates(subset='ordered_pair').drop(columns='ordered_pair')

# Filtrar pelas correlações fortes (acima de 0.5 ou abaixo de -0.5)
strong_corr = correlation_table[
    (correlation_table['Correlation'] > 0.5) | (correlation_table['Correlation'] < -0.5)
].sort_values(by='Correlation', ascending=False)

strong_corr

,Feature 1,Feature 2,Correlation
711,atv_cultural_apresentacoes musicais,atv_culturais_a_fazer_apresentacoes musicais,1.000000
14149,jogo_resta um,lutas_desejadas_taekwondo,1.000000
15883,luta_jeet kune do,projetos_a_participar_time de e-sports do camp...,1.000000
19424,atv_culturais_a_fazer_oficina de gastronomia,transporte_mais onibus,1.000000
21466,hab_musicais_desejadas_kalimba,melhoria_estrutura_colocar mais bancos e subst...,1.000000
...,...,...,...
21003,hab_musicais_desejadas_instrumentos de corda,hab_musicais_desejadas_nenhuma opcao,-0.606214
17011,luta_muay thai,luta_nenhuma opcao,-0.613027
5674,hab_musical_canto,hab_musical_nenhuma opcao,-0.617654
42634,projetos_a_participar_nenhuma opcao,projetos_a_participar_recuperacao e ocupacao d...,-0.638056
